# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [3]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

#### Regresión logística

In [6]:
# build a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9,solver='liblinear',multi_class='ovr')
logreg.fit(X_train, y_train)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [7]:
y_pred = logreg.predict(X_test)

In [8]:
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred))
print(metrics.f1_score(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

0.6213333333333333
0.6033519553072626
             precision    recall  f1-score   support

          0       0.61      0.67      0.64       744
          1       0.64      0.57      0.60       756

avg / total       0.62      0.62      0.62      1500



In [9]:
# instantiate a DecisionTreeRegressor (with random_state=1)
from sklearn.tree import DecisionTreeClassifier
treeclas = DecisionTreeClassifier(random_state=1)
treeclas.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [10]:
y_pred = treeclas.predict(X_test)

In [11]:
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred))
print(metrics.f1_score(y_test,y_pred))
print(metrics.classification_report(y_test,y_pred))

0.5426666666666666
0.5402144772117962
             precision    recall  f1-score   support

          0       0.54      0.55      0.55       744
          1       0.55      0.53      0.54       756

avg / total       0.54      0.54      0.54      1500



# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

### Decision tree - max depth none

In [12]:
import numpy as np

# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]
n_B = 100

# create 100 bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

In [13]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
dtc1 = DecisionTreeClassifier(max_depth=None, random_state=123)

# DataFrame for storing predicted price from each tree
y1_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X1_train = X_train.iloc[sample]
    y1_train = y_train.iloc[sample]
    dtc1.fit(X1_train, y1_train)
    y1_pred[i] = dtc1.predict(X_test)

In [15]:
y1_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1483,1,1,0,0,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,0,1,0,1,1,0,...,1,0,1,1,0,1,1,1,1,1
2520,1,0,0,0,1,1,1,1,0,0,...,1,1,1,0,1,0,1,0,1,1
3721,1,0,1,0,1,1,1,0,1,1,...,1,1,1,1,0,1,0,1,1,1
3727,0,0,0,0,1,0,1,0,0,0,...,0,0,1,1,0,0,0,0,1,0


### Decision tree - max depth 2

In [16]:
np.random.seed(123)

# create 100 bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

In [17]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
dtc2 = DecisionTreeClassifier(max_depth=2, random_state=123)

# DataFrame for storing predicted price from each tree
y2_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X2_train = X_train.iloc[sample]
    y2_train = y_train.iloc[sample]
    dtc2.fit(X2_train, y2_train)
    y2_pred[i] = dtc2.predict(X_test)

In [19]:
y2_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1483,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3721,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Logistic regression

In [20]:
np.random.seed(123)

# create 100 bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

In [21]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
lr = LogisticRegression(solver='liblinear',C=1e9, random_state=123)

# DataFrame for storing predicted price from each tree
y3_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])

# grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X3_train = X_train.iloc[sample]
    y3_train = y_train.iloc[sample]
    lr.fit(X3_train, y3_train)
    y3_pred[i] = lr.predict(X_test)

In [22]:
y3_pred.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1483,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2185,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3721,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [23]:
y1_pred=y1_pred.mode(axis=1)[0]
y2_pred=y2_pred.mode(axis=1)[0]
y3_pred=y3_pred.mode(axis=1)[0]

In [24]:
from sklearn import metrics
print(metrics.f1_score(y1_pred, y_test))
print(metrics.f1_score(y2_pred, y_test))
print(metrics.f1_score(y3_pred, y_test))

0.6463576158940397
0.6448598130841121
0.6090592334494773


In [25]:
mv_pred = pd.concat([y1_pred, y2_pred,y3_pred], axis=1, keys=['dtc1', 'dtc2','lr'])
mv_pred = mv_pred.mode(axis=1)
mv_pred = mv_pred.iloc[:,0]

In [27]:
from sklearn import metrics
metrics.f1_score(mv_pred, y_test)

0.6405405405405405

### Mayority voting con sklearn

In [52]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

dtc1 = DecisionTreeClassifier(max_depth=None, random_state=125)
dtc2 = DecisionTreeClassifier(max_depth=2, random_state=125)
lr = LogisticRegression(solver='liblinear',C=1e9, random_state=125)

bagdtc1 = BaggingClassifier(dtc1, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)
bagdtc2 = BaggingClassifier(dtc2, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)
baglr = BaggingClassifier(lr, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)

bagclas = VotingClassifier(estimators=[('dtc1',dtc1),('dtc2',dtc2),('lr',lr)], voting='hard')
bagclas.fit(X_train,y_train)
mv2_pred = bagclas.predict(X_test)

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [53]:
from sklearn import metrics
metrics.f1_score(mv2_pred, y_test)

0.6367346938775511

# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [55]:
mv2_pred = pd.DataFrame(mv2_pred)
mv2_pred = mv2_pred.iloc[:,0]
mv2_pred.value_counts().to_frame('count').assign(percentage = lambda x: x/x.sum())

,count,percentage
0,786,0.524
1,714,0.476


La probabilidad de que el modelo clasifique un evento como positivo (1) es de 0.47

In [57]:
threshold = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
th_pred1 = pd.DataFrame(index=y_test.index, columns= threshold)
f1_1 = pd.DataFrame(index = ['f1'], columns = threshold)

bagdtc1.fit(X_train, y_train)

for i in threshold:
    th_pred1[i] = (bagdtc1.predict_proba(X_test)[:,1] >= i).astype(np.int)
    f1_1[i]=metrics.f1_score(th_pred1[i], y_test)
f1_1

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
f1,0.673524,0.68331,0.701183,0.701156,0.66105,0.549839,0.338493,0.133011,0.013089


El threshold que maximiza f1 para el árbol sin numero máximo de niveles es 0.3

In [58]:
threshold = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
th_pred2 = pd.DataFrame(index=y_test.index, columns= threshold)
f1_2 = pd.DataFrame(index = ['f1'], columns = threshold)

bagdtc2.fit(X_train, y_train)

for i in threshold:
    th_pred2[i] = (bagdtc2.predict_proba(X_test)[:,1] >= i).astype(np.int)
    f1_2[i]=metrics.f1_score(th_pred2[i], y_test)
f1_2

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
f1,0.670213,0.670213,0.678088,0.701068,0.644772,0.558685,0.188153,0.0,0.0


El threshold que maximiza f1 para el árbol con 2 niveles de profundida máxima es 0.4

In [59]:
threshold = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
th_pred3 = pd.DataFrame(index=y_test.index, columns= threshold)
f1_3 = pd.DataFrame(index = ['f1'], columns = threshold)

baglr.fit(X_train, y_train)

for i in threshold:
    th_pred3[i] = (baglr.predict_proba(X_test)[:,1] >= i).astype(np.int)
    f1_3[i]=metrics.f1_score(th_pred3[i], y_test)
f1_3

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
f1,0.670213,0.66933,0.676866,0.686243,0.606398,0.485915,0.288191,0.09901,0.015686


El threshold que maximiza f1 para la regresión logística es 0.4

In [73]:
mv_final = pd.concat([th_pred1.iloc[:,2], th_pred2.iloc[:,3],th_pred3.iloc[:,3]], axis=1, keys=['dtc1', 'dtc2','lr'])
mv_final = mv_final.mode(axis=1)
mv_final = mv_final.iloc[:,0]

In [75]:
from sklearn import metrics
metrics.f1_score(mv_final, y_test)

0.7022411953041622

El f1 score aumenta significativamente cuando se toma la predicción con el threshold que maximiza el f1 en cada modelo

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [269]:
dtc1 = DecisionTreeClassifier(max_depth=None, random_state=125)
dtc2 = DecisionTreeClassifier(max_depth=2, random_state=125)
lr = LogisticRegression(solver='liblinear',C=1e9, random_state=125)

bagdtc1 = BaggingClassifier(dtc1, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)
bagdtc2 = BaggingClassifier(dtc2, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)
baglr = BaggingClassifier(lr, n_estimators=100,bootstrap=True, oob_score=True,n_jobs=-1)

bagclas = VotingClassifier(estimators=[('dtc1',dtc1),('dtc2',dtc2),('lr',lr)], voting='soft')
bagclas.fit(X_train,y_train)
wv2_pred = bagclas.predict(X_test)

C:\Users\nicol\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [266]:
from sklearn import metrics
metrics.f1_score(wv2_pred, y_test)

0.5540897097625331

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [273]:
wv2_pred = pd.DataFrame(wv2_pred)
wv2_pred = wv2_pred.iloc[:,0]
wv2_pred.value_counts().to_frame('count').assign(percentage = lambda x: x/x.sum())

,count,percentage
1,760,0.506667
0,740,0.493333


La probabilidad de que el modelo clasifique un evento como positivo (1) es de 0.49

In [274]:
threshold = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
th_pred = pd.DataFrame(index=y_test.index, columns= threshold)
f1 = pd.DataFrame(index = ['f1'], columns = threshold)

for i in threshold:
    th_pred[i] = (bagclas.predict_proba(X_test)[:,1] >= i).astype(np.int)
    f1[i]=metrics.f1_score(th_pred[i], y_test)
f1

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
f1,0.670213,0.674567,0.669935,0.614241,0.55409,0.538799,0.413479,0.204833,0.002642


El threshold que maximiza f1 es 0.2

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score